# Importing Necessary Packages

In [16]:
%load_ext autoreload
%autoreload 2
    
# Importing all the functions I made for data creation/manipulation
from data_creation import data_creation, pickle_me, get_pickle

# Model Building
import pandas as pd
import numpy as np

# Neural Network Building
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import regularizers
from keras.layers import Dense, Flatten, Dropout, Conv1D, Conv2D, ELU

# Classification Metrics
from sklearn.metrics import confusion_matrix
from keras.metrics import Precision, Recall
from data_creation import get_precisions
from sklearn.metrics import precision_score, recall_score, f1_score

# Packaging Data
import pickle

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Creation

#### I only need to run this once to create my data. It will pickle my data for future reference.
I will only run the categories object since I call on it later in the notebook.

In [19]:
# Directory where all the images are stored.
#data_dir = 'C://Users/Cristian/Documents/FlatIron/Capstone/Fruit'

# Categories of Fruit with the correct labels.
categories = ['Apple', 'Banana', 'Carambola', 'Guava', 'Kiwi', 'Mango',
              'Muskmelon', 'Orange', 'Peach', 'Pear', 'Persimmon', 'Pitaya',
              'Plum', 'Pomegranate', 'Tomato']

##### Creating Grayscale data and pickling it

In [21]:
#X_gray, y_gray = data_creation(50, 0, categories, data_dir)
#pickle_me(X_gray,y_gray, 'X_grayscale', 'y_grayscale')

##### Creating Color data and pickling it

In [22]:
#X_color, y_color = data_creation(50, 1, categories, data_dir)
#pickle_me(X_color, y_color,'X_color', 'y_color')

##### Getting our pickled data for grayscale images and color images

In [23]:
X_grayscale, y_grayscale = get_pickle('X_grayscale.pickle', 'y_grayscale.pickle')

In [24]:
X_rgb, y_rgb = get_pickle('X_color.pickle', 'y_color.pickle')

# Data Manipulation

#### Creating a train, test split for cross-validation.
'gs' will be for grayscale images, 'rgb' will be for color images.

I will rescale by 255 as a standard procedure for all my images and reshape
them to the correct input I need.

I need a target input shape of (15,) instead of (1,), which is why I use
get_dummies. This will create a column for each label with entries as a binary
input.

In [25]:
X_rgb_sc = X_rgb/255.0
X_rgb_reshape = X_rgb_sc.reshape(X_rgb.shape[0],50,50,3)

In [26]:
X_gs_sc = X_grayscale/255.0
X_gs_reshape = X_gs_sc.reshape(X_grayscale.shape[0],50,50,)

In [27]:
X_gs_train, X_gs_test, y_gs_train, y_gs_test = train_test_split(X_gs_reshape,
                                                                y_grayscale, 
                                                                test_size=.2,
                                                                random_state=42)

In [28]:
X_rgb_train, X_rgb_test, y_rgb_train, y_rgb_test = train_test_split(X_rgb_reshape,
                                                                    y_rgb, 
                                                                    test_size=.2,
                                                                    random_state=42)

In [29]:
y_gs_train_dummies = np.array(pd.get_dummies(y_gs_train))
y_gs_test_dummies = np.array(pd.get_dummies(y_gs_test))

y_rgb_train_dummies = np.array(pd.get_dummies(y_rgb_train))
y_rgb_test_dummies = np.array(pd.get_dummies(y_rgb_test))

# Model Building
As a way to better visualize how models are doing compared to one another, I will create the neural networks, compile them, and fit our data. Then, I will have them predict classes and put our classification metrics into a dataframe. You will notice that these models are very simple. I had already tried adding more layers without much progress. Keeping the model to only a few layers big could achieve similar or even better results that a more complex and larger model.

## Grayscale Models

### Model 1
First Simple Model
Im using a Flatten layer so I can input my data through a Dense layer where softmax will split into the 15 classifications I have for the
fruit.

In [30]:
model_1 = Sequential([Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_1.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_1_results = model_1.fit(X_gs_train, y_gs_train_dummies, epochs=5, validation_data =(X_gs_test, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 2s 50us/step - loss: 1.6998 - categorical_accuracy: 0.5456 - precision_2: 0.8920 - recall_2: 0.2309 - val_loss: 1.5539 - val_categorical_accuracy: 0.5345 - val_precision_2: 0.7709 - val_recall_2: 0.2981
Epoch 2/5
35524/35524 [==============================] - 2s 49us/step - loss: 1.2124 - categorical_accuracy: 0.6841 - precision_2: 0.9507 - recall_2: 0.4619 - val_loss: 1.3410 - val_categorical_accuracy: 0.6306 - val_precision_2: 0.8178 - val_recall_2: 0.4447
Epoch 3/5
35524/35524 [==============================] - 2s 49us/step - loss: 1.1055 - categorical_accuracy: 0.7124 - precision_2: 0.9555 - recall_2: 0.5361 - val_loss: 1.1070 - val_categorical_accuracy: 0.7105 - val_precision_2: 0.9452 - val_recall_2: 0.5365
Epoch 4/5
35524/35524 [==============================] - 2s 48us/step - loss: 1.0441 - categorical_accuracy: 0.7246 - precision_2: 0.9594 - recall_2: 0.5745 - val_loss: 1.

### Model 2
Adding a convolutional layer to try to improve performance.

In [31]:
model_2 = Sequential([Conv1D(100, kernel_size=(1), activation='relu', input_shape=(50,50)),
                      Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_2.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_2_results = model_2.fit(X_gs_train, y_gs_train_dummies, epochs=5, validation_data =(X_gs_test, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 4s 111us/step - loss: 1.1383 - categorical_accuracy: 0.6886 - precision_3: 0.9422 - recall_3: 0.5336 - val_loss: 1.1407 - val_categorical_accuracy: 0.6584 - val_precision_3: 0.8195 - val_recall_3: 0.5645
Epoch 2/5
35524/35524 [==============================] - 4s 107us/step - loss: 0.8056 - categorical_accuracy: 0.7742 - precision_3: 0.9606 - recall_3: 0.6891 - val_loss: 0.8742 - val_categorical_accuracy: 0.7472 - val_precision_3: 0.8977 - val_recall_3: 0.6759
Epoch 3/5
35524/35524 [==============================] - 4s 105us/step - loss: 0.7346 - categorical_accuracy: 0.7878 - precision_3: 0.9649 - recall_3: 0.7143 - val_loss: 0.7863 - val_categorical_accuracy: 0.7798 - val_precision_3: 0.9405 - val_recall_3: 0.7177
Epoch 4/5
35524/35524 [==============================] - 4s 105us/step - loss: 0.6959 - categorical_accuracy: 0.7981 - precision_3: 0.9665 - recall_3: 0.7264 - val_loss

### Model 3
Changing the number of neurons for the convolutional layer to see if less neurons can have an improvement.

In [32]:
model_3 = Sequential([Conv1D(50, kernel_size=(1), activation='relu', input_shape=(50,50)),
                      Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_3.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_3_results = model_3.fit(X_gs_train, y_gs_train_dummies, epochs=5, validation_data =(X_gs_test, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 3s 86us/step - loss: 1.2106 - categorical_accuracy: 0.6696 - precision_4: 0.9400 - recall_4: 0.4887 - val_loss: 1.0426 - val_categorical_accuracy: 0.7175 - val_precision_4: 0.8863 - val_recall_4: 0.6098
Epoch 2/5
35524/35524 [==============================] - 3s 97us/step - loss: 0.8952 - categorical_accuracy: 0.7544 - precision_4: 0.9529 - recall_4: 0.6515 - val_loss: 0.9702 - val_categorical_accuracy: 0.7207 - val_precision_4: 0.9328 - val_recall_4: 0.6237
Epoch 3/5
35524/35524 [==============================] - 3s 91us/step - loss: 0.8246 - categorical_accuracy: 0.7707 - precision_4: 0.9596 - recall_4: 0.6827 - val_loss: 0.8477 - val_categorical_accuracy: 0.7590 - val_precision_4: 0.9522 - val_recall_4: 0.6779
Epoch 4/5
35524/35524 [==============================] - 3s 89us/step - loss: 0.7860 - categorical_accuracy: 0.7784 - precision_4: 0.9611 - recall_4: 0.6979 - val_loss: 0.

### Model 4
Adding another dense layer with another activation function. I had tested out several activation functions that all tended to
either stay stagnant or decrease performance.

In [33]:
model_4 = Sequential([Conv1D(50, kernel_size=(1), activation='relu', input_shape=(50,50)),
                      Flatten(input_shape=(50,50)),
                      Dense(30, activation='sigmoid'),
                      Dense(15, activation='softmax')])
model_4.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_4_results = model_4.fit(X_gs_train, y_gs_train_dummies, epochs=5, validation_data =(X_gs_test, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 4s 99us/step - loss: 1.6298 - categorical_accuracy: 0.5934 - precision_5: 0.9906 - recall_5: 0.1713 - val_loss: 1.3897 - val_categorical_accuracy: 0.6540 - val_precision_5: 0.9862 - val_recall_5: 0.2650
Epoch 2/5
35524/35524 [==============================] - 3s 97us/step - loss: 1.1926 - categorical_accuracy: 0.7087 - precision_5: 0.9829 - recall_5: 0.4427 - val_loss: 1.1544 - val_categorical_accuracy: 0.7091 - val_precision_5: 0.9823 - val_recall_5: 0.4814
Epoch 3/5
35524/35524 [==============================] - 3s 95us/step - loss: 1.0537 - categorical_accuracy: 0.7331 - precision_5: 0.9811 - recall_5: 0.5457 - val_loss: 1.0242 - val_categorical_accuracy: 0.7336 - val_precision_5: 0.9837 - val_recall_5: 0.5760
Epoch 4/5
35524/35524 [==============================] - 3s 96us/step - loss: 0.9734 - categorical_accuracy: 0.7470 - precision_5: 0.9795 - recall_5: 0.6006 - val_loss: 0.

### Model 5
Adding a dropout layer. This would make sure that we can prevent our model from overrelying on certain features to make predictions.

In [34]:
model_5 = Sequential([Conv1D(50, kernel_size=(1), activation='relu', input_shape=(50,50)),
                      Flatten(input_shape=(50,50)),
                      Dropout(0.2),
                      Dense(30, activation='sigmoid'),
                      Dense(15, activation='softmax')])
model_5.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_5_results = model_5.fit(X_gs_train, y_gs_train_dummies, epochs=5, validation_data =(X_gs_test, y_gs_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 5s 134us/step - loss: 1.6832 - categorical_accuracy: 0.5847 - precision_6: 0.9877 - recall_6: 0.1248 - val_loss: 1.3758 - val_categorical_accuracy: 0.6816 - val_precision_6: 0.9948 - val_recall_6: 0.3003
Epoch 2/5
35524/35524 [==============================] - 4s 124us/step - loss: 1.2485 - categorical_accuracy: 0.7039 - precision_6: 0.9807 - recall_6: 0.4114 - val_loss: 1.1938 - val_categorical_accuracy: 0.6994 - val_precision_6: 0.9725 - val_recall_6: 0.4619
Epoch 3/5
35524/35524 [==============================] - 4s 123us/step - loss: 1.1081 - categorical_accuracy: 0.7285 - precision_6: 0.9771 - recall_6: 0.5202 - val_loss: 1.0592 - val_categorical_accuracy: 0.7453 - val_precision_6: 0.9854 - val_recall_6: 0.5551
Epoch 4/5
35524/35524 [==============================] - 4s 122us/step - loss: 1.0258 - categorical_accuracy: 0.7439 - precision_6: 0.9757 - recall_6: 0.5754 - val_loss

## Color Images Model

### Model 6
We are starting to use color images since color is a big factor in determing fruit apart. 

Model 6 will be first simple model for color images.

In [53]:
model_6 = Sequential([Conv2D(30, kernel_size=(1), activation='relu'),
                      Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_6.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_6_results = model_6.fit(X_rgb_train, y_rgb_train_dummies, epochs=5, validation_data =(X_rgb_test, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 26s 745us/step - loss: 0.8948 - categorical_accuracy: 0.8172 - precision_11: 0.9021 - recall_11: 0.7475 - val_loss: 0.4729 - val_categorical_accuracy: 0.8488 - val_precision_11: 0.9114 - val_recall_11: 0.8024
Epoch 2/5
35524/35524 [==============================] - 26s 735us/step - loss: 0.2825 - categorical_accuracy: 0.9130 - precision_11: 0.9563 - recall_11: 0.8723 - val_loss: 0.2957 - val_categorical_accuracy: 0.9071 - val_precision_11: 0.9457 - val_recall_11: 0.8753
Epoch 3/5
35524/35524 [==============================] - 26s 728us/step - loss: 0.2236 - categorical_accuracy: 0.9331 - precision_11: 0.9660 - recall_11: 0.8986 - val_loss: 0.2613 - val_categorical_accuracy: 0.9150 - val_precision_11: 0.9487 - val_recall_11: 0.8847
Epoch 4/5
35524/35524 [==============================] - 26s 723us/step - loss: 0.1911 - categorical_accuracy: 0.9453 - precision_11: 0.9724 - recall_11:

### Model 7

Changing the activation function to see if there is a difference and lowering the neurons.

In [36]:
model_7 = Sequential([Conv2D(50, kernel_size=(1), activation='sigmoid'),
                      Flatten(input_shape=(50,50)),
                      Dense(15, activation='softmax')])
model_7.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_7_results = model_7.fit(X_rgb_train, y_rgb_train_dummies, epochs=5, validation_data =(X_rgb_test, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 44s 1ms/step - loss: 47.3547 - categorical_accuracy: 0.1138 - precision_8: 0.1148 - recall_8: 0.1090 - val_loss: 15.5324 - val_categorical_accuracy: 0.1306 - val_precision_8: 0.1307 - val_recall_8: 0.1304
Epoch 2/5
35524/35524 [==============================] - 44s 1ms/step - loss: 6.7625 - categorical_accuracy: 0.2686 - precision_8: 0.2953 - recall_8: 0.2396 - val_loss: 11.4394 - val_categorical_accuracy: 0.0706 - val_precision_8: 0.0694 - val_recall_8: 0.0691
Epoch 3/5
35524/35524 [==============================] - 44s 1ms/step - loss: 3.2889 - categorical_accuracy: 0.4083 - precision_8: 0.4764 - recall_8: 0.3579 - val_loss: 2.3299 - val_categorical_accuracy: 0.4331 - val_precision_8: 0.4777 - val_recall_8: 0.3781
Epoch 4/5
35524/35524 [==============================] - 44s 1ms/step - loss: 2.3089 - categorical_accuracy: 0.4925 - precision_8: 0.5838 - recall_8: 0.4322 - val_loss:

### Model 8

Adding a dropout layer for the same reason as before. Lowering number of neurons as well. My valudation values for accuracy, precision, and recall seemed to suffer after 5 epocs, so I lowered it to 4.

In [12]:
model_8 = Sequential([Conv2D(85, kernel_size=(1), activation='relu'),
                      Flatten(input_shape=(50,50)),
                      Dropout(0.2),
                      Dense(15, activation='softmax')])
model_8.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_8_results = model_8.fit(X_rgb_train, y_rgb_train_dummies, epochs=4, validation_data =(X_rgb_test, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/4
35524/35524 [==============================] - 121s 3ms/step - loss: 1.0381 - categorical_accuracy: 0.8366 - precision_1: 0.9087 - recall_1: 0.7756 - val_loss: 0.3072 - val_categorical_accuracy: 0.9032 - val_precision_1: 0.9449 - val_recall_1: 0.8705
Epoch 2/4
35524/35524 [==============================] - 121s 3ms/step - loss: 0.2267 - categorical_accuracy: 0.9291 - precision_1: 0.9619 - recall_1: 0.8962 - val_loss: 0.2376 - val_categorical_accuracy: 0.9228 - val_precision_1: 0.9566 - val_recall_1: 0.8954
Epoch 3/4
35524/35524 [==============================] - 121s 3ms/step - loss: 0.1718 - categorical_accuracy: 0.9501 - precision_1: 0.9730 - recall_1: 0.9240 - val_loss: 0.2075 - val_categorical_accuracy: 0.9308 - val_precision_1: 0.9599 - val_recall_1: 0.9059
Epoch 4/4
35524/35524 [==============================] - 120s 3ms/step - loss: 0.1392 - categorical_accuracy: 0.9613 - precision_1: 0.9792 - recall_1: 0.9399 - val_loss

#### Model 9
Adding bias regularizers and lowering the neurons for the convolution layer.

In [37]:
model_9 = Sequential([Conv2D(10, kernel_size=(1), activation='relu'),
                      Flatten(input_shape=(50,50)),
                      Dropout(0.2),
                      Dense(64, activation='sigmoid', bias_regularizer=regularizers.l1_l2(l1=0.02,l2=0.02)),
                      Dense(15, activation='softmax')])
model_9.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_9_results = model_9.fit(X_rgb_train, y_rgb_train_dummies, epochs=5, validation_data =(X_rgb_test, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 29s 813us/step - loss: 1.0007 - categorical_accuracy: 0.7384 - precision_9: 0.9717 - recall_9: 0.5128 - val_loss: 0.6704 - val_categorical_accuracy: 0.8250 - val_precision_9: 0.9597 - val_recall_9: 0.7032
Epoch 2/5
35524/35524 [==============================] - 29s 804us/step - loss: 0.5340 - categorical_accuracy: 0.8575 - precision_9: 0.9685 - recall_9: 0.7652 - val_loss: 0.5844 - val_categorical_accuracy: 0.8311 - val_precision_9: 0.9097 - val_recall_9: 0.7694
Epoch 3/5
35524/35524 [==============================] - 29s 809us/step - loss: 0.4423 - categorical_accuracy: 0.8754 - precision_9: 0.9674 - recall_9: 0.8019 - val_loss: 0.4808 - val_categorical_accuracy: 0.8619 - val_precision_9: 0.9584 - val_recall_9: 0.7833
Epoch 4/5
35524/35524 [==============================] - 29s 805us/step - loss: 0.3899 - categorical_accuracy: 0.8892 - precision_9: 0.9684 - recall_9: 0.8211 - val_

### Model 10
Similar to model 8, but I'm changing the order of the layers and added a regularizer.

In [38]:
model_10 = Sequential([Conv2D(85, kernel_size=(1), activation='relu'),
                       Dropout(0.2),
                       Flatten(input_shape=(50,50)),
                       Dense(15, activation='softmax', activity_regularizer=regularizers.l1(0.02))])
model_10.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy', Precision(), Recall()])
model_10_results = model_10.fit(X_rgb_train, y_rgb_train_dummies, epochs=5, validation_data =(X_rgb_test, y_rgb_test_dummies))

Train on 35524 samples, validate on 8882 samples
Epoch 1/5
35524/35524 [==============================] - 127s 4ms/step - loss: 1.5750 - categorical_accuracy: 0.8464 - precision_10: 0.9135 - recall_10: 0.7885 - val_loss: 1.0301 - val_categorical_accuracy: 0.8780 - val_precision_10: 0.9288 - val_recall_10: 0.8385
Epoch 2/5
35524/35524 [==============================] - 127s 4ms/step - loss: 0.8591 - categorical_accuracy: 0.9351 - precision_10: 0.9651 - recall_10: 0.9023 - val_loss: 1.0874 - val_categorical_accuracy: 0.8506 - val_precision_10: 0.8838 - val_recall_10: 0.8247
Epoch 3/5
35524/35524 [==============================] - 127s 4ms/step - loss: 0.8042 - categorical_accuracy: 0.9523 - precision_10: 0.9732 - recall_10: 0.9286 - val_loss: 0.8473 - val_categorical_accuracy: 0.9349 - val_precision_10: 0.9588 - val_recall_10: 0.9113
Epoch 4/5
35524/35524 [==============================] - 127s 4ms/step - loss: 0.7714 - categorical_accuracy: 0.9641 - precision_10: 0.9812 - recall_10: 0.9

# Getting Metrics for my top 2 Models

My goal is to get the most amount of correct predictions. This means that of all the predictions I make, I want to get the most amount possible correct. This is similar to the precision metric for classification. Thus, I will first look at how these models do with precision. I will still look at recall and f1-score to get a sense on how these models are doing.

### Closer Look

In [44]:
models = [model_1, model_2, model_3, model_4, model_5,
         model_6, model_7, model_8, model_9, model_10]

In [45]:
model_8_predictions = model_8.predict_classes(X_rgb_test)
model_10_predictions = model_10.predict_classes(X_rgb_test)

### Precision

In [46]:
model_8_precision = precision_score(y_rgb_test, model_8_predictions, average='weighted')
model_10_precision = precision_score(y_rgb_test, model_10_predictions, average='weighted')
print('Model 8', model_8_precision)
print('Model 10:', model_10_precision)

Model 8 0.9336262940958922
Model 10: 0.9425046136065578


### Recall

In [47]:
model_8_recall = recall_score(y_rgb_test, model_8_predictions, average='weighted')
model_10_recall = recall_score(y_rgb_test, model_10_predictions, average='weighted')
print('Model 8:', model_8_recall)
print('Model 10:', model_10_recall)

Model 8: 0.9296329655482999
Model 10: 0.9412294528259401


### F1-Score

In [48]:
model_8_f1 = f1_score(y_rgb_test, model_8_predictions, average='weighted')
model_10_f1 = f1_score(y_rgb_test, model_10_predictions, average='weighted')
print('Model 6:', model_8_f1)
print('Model 10:', model_10_f1)

Model 6: 0.9305013142723558
Model 10: 0.9415362706726801


## Conclusion

Model 8 and Model 10 are our top 2 models based on values in validation accuracy, precision, recall, and f1-score. However, I am choosing Model 8 as my main model even though the precision isn't as high, but still very similar to Model 10. They each have very similar scores in everything except loss. When it comes to loss, model 8 has a much lower crossentropy loss. This means the distribution of model 8's predictions are closer to the distribution of the actual labels.

#### Save my models
This will save each model architecture as a .json file with the weights of the features as a .h5 file. This will let me save all of my models to save me time in the future.

In [49]:
for model in models:
    i = models.index(model) + 1
    model_json = model.to_json()
    with open(f"Models/model{i}.json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(f"Models/model_weights_{i}.h5")

#### Get Scores for all of my Data
I am using a weighted average since it takes into account the slight imbalance of the data.

In [50]:
m8_final_preds = model_8.predict_classes(X_rgb_reshape)
prec = precision_score(y_rgb, m8_final_preds, average='weighted')
rec = recall_score(y_rgb, m8_final_preds, average='weighted')
f1 = f1_score(y_rgb, m8_final_preds, average='weighted')
print('Precision:', prec,
      '\nRecall:', rec,
      '\nF1-Score:', f1)

Precision: 0.9633755541842637 
Recall: 0.9613790929153718 
F1-Score: 0.9617900240187346


## Packaging Predictions for EDA

In [51]:
predictions_df = pd.DataFrame(list(zip(y_rgb, m8_final_preds)))
predictions_df.columns = ['Actual', 'Predicted']

In [52]:
predictions_pickle_out = open('model_predictions.pickle', 'wb')
pickle.dump(predictions_df, predictions_pickle_out)
predictions_pickle_out.close()